In [1]:
# Cell 1: Import required libraries
import os
import cv2
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision.transforms as transforms
import timm

from sklearn.metrics import roc_auc_score
from scipy.ndimage import gaussian_filter, map_coordinates
import random

import time

from sklearn.preprocessing import MultiLabelBinarizer


In [2]:
# Cell 2: Define custom augmentation transforms

# CLAHE Transform for grayscale images
class CLAHETransform:
    def __init__(self, clipLimit=2.0, tileGridSize=(8, 8)):
        self.clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)

    def __call__(self, img):
        img_np = np.array(img)
        # For grayscale images, apply CLAHE directly
        img_np = self.clahe.apply(img_np)
        return Image.fromarray(img_np)

# Gamma Correction Transform
class GammaCorrectionTransform:
    def __init__(self, gamma=1.0):
        self.gamma = gamma

    def __call__(self, img):
        img_np = np.array(img).astype(np.float32) / 255.0
        corrected = np.power(img_np, self.gamma)
        corrected = np.clip(corrected * 255, 0, 255).astype(np.uint8)
        return Image.fromarray(corrected)

# Elastic Deformation Transform
class ElasticTransform:
    def __init__(self, alpha=34, sigma=4, random_state=None):
        self.alpha = alpha
        self.sigma = sigma
        self.random_state = random_state if random_state is not None else np.random.RandomState(None)

    def __call__(self, img):
        img_np = np.array(img)
        shape = img_np.shape[:2]
        
        dx = gaussian_filter((self.random_state.rand(*shape) * 2 - 1), self.sigma, mode="constant", cval=0) * self.alpha
        dy = gaussian_filter((self.random_state.rand(*shape) * 2 - 1), self.sigma, mode="constant", cval=0) * self.alpha
        
        x, y = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]))
        indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1))
        
        if img_np.ndim == 3:
            channels = []
            for i in range(img_np.shape[2]):
                channel = map_coordinates(img_np[..., i], indices, order=1, mode='reflect').reshape(shape)
                channels.append(channel)
            transformed = np.stack(channels, axis=2)
        else:
            transformed = map_coordinates(img_np, indices, order=1, mode='reflect').reshape(shape)
        return Image.fromarray(transformed.astype(np.uint8))


In [3]:
# Cell 3: Define Focal Loss for multi-label classification
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        """
        Args:
            alpha (float): Balancing factor.
            gamma (float): Focusing parameter.
            reduction (str): Reduction method ('mean', 'sum', or 'none').
        """
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        
    def forward(self, inputs, targets):
        # Compute binary cross entropy loss per element
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss
        
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

# Define the 15 classes (14 diseases plus "No Finding")
CLASSES = [
    "No Finding", "Atelectasis", "Cardiomegaly", "Effusion", "Infiltration", 
    "Mass", "Nodule", "Pneumonia", "Pneumothorax", "Consolidation", 
    "Edema", "Emphysema", "Fibrosis", "Pleural_Thickening", "Hernia"
]

# Load CSV data
df = pd.read_csv("/student/csc490_project/shared/labels.csv")

# Process the "Finding Labels" column.
# For "No Finding", this will return ["No Finding"],
# and for multiple findings it splits by '|'.
def process_labels(label_str):
    return label_str.split("|")

df["label_list"] = df["Finding Labels"].apply(process_labels)

# Initialize and apply MultiLabelBinarizer with our defined classes.
mlb = MultiLabelBinarizer(classes=CLASSES)
labels_array = mlb.fit_transform(df["label_list"])

# Save the multi-hot vectors into a new column.
df["labels"] = list(labels_array)

# Get the unique patient IDs from the dataset
unique_patients = df["Patient ID"].unique()

# Shuffle the patient IDs using a fixed random seed for reproducibility
np.random.seed(42)
np.random.shuffle(unique_patients)

# Determine the split index for a 70/30 split (by patient)
split_index = int(0.7 * len(unique_patients))
train_patients = unique_patients[:split_index]
val_patients = unique_patients[split_index:]

# Filter the dataframe based on the patient IDs for each split
train_df = df[df["Patient ID"].isin(train_patients)].reset_index(drop=True)
val_df = df[df["Patient ID"].isin(val_patients)].reset_index(drop=True)

print(f"Training samples: {len(train_df)}, Validation samples: {len(val_df)}")

# Print unique patient counts for each split to verify no overlap
print(f"Unique patients in training: {len(train_df['Patient ID'].unique())}")
print(f"Unique patients in validation: {len(val_df['Patient ID'].unique())}")

# Verify there is no overlap in patient IDs between the splits
overlap_ids = set(train_df["Patient ID"].unique()).intersection(val_df["Patient ID"].unique())
if overlap_ids:
    print("Overlap in Patient IDs found:", overlap_ids)
else:
    print("No overlap in Patient IDs between training and validation sets.")

# Define the dataset class using the precomputed multi-hot labels.
class ChestXrayDataset(Dataset):
    def __init__(self, df, root_dir, transform=None):
        """
        Args:
            df: DataFrame with columns "Image Index" and "labels".
            root_dir: Directory where images are stored.
            transform: Transformations to apply.
        """
        self.df = df
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # Load the image in grayscale.
        img_name = os.path.join(self.root_dir, self.df.iloc[idx]["Image Index"])
        image = Image.open(img_name).convert("L")
        
        # Retrieve the multi-hot label vector (length 15).
        labels = np.array(self.df.iloc[idx]["labels"], dtype=np.float32)
        labels = torch.tensor(labels, dtype=torch.float)
        
        if self.transform:
            image = self.transform(image)
        return image, labels


Training samples: 78708, Validation samples: 33412
Unique patients in training: 21563
Unique patients in validation: 9242
No overlap in Patient IDs between training and validation sets.


In [5]:
# Cell 5: Record split index

# Print dataset sizes
print(f"Total samples: {len(df)}")
print(f"Training samples: {len(train_df)}, Validation samples: {len(val_df)}")
print(f"Split index: {split_index}")

# Print first 5 indices of each split to verify consistency
print("First 5 indices in training set:")
print(train_df.index[:5].tolist())

print("First 5 indices in validation set:")
print(val_df.index[:5].tolist())

# Print first few image filenames in each split
print("First 5 images in training set:")
print(train_df["Image Index"].head().tolist())

print("First 5 images in validation set:")
print(val_df["Image Index"].head().tolist())

Total samples: 112120
Training samples: 78708, Validation samples: 33412
Split index: 21563
First 5 indices in training set:
[0, 1, 2, 3, 4]
First 5 indices in validation set:
[0, 1, 2, 3, 4]
First 5 images in training set:
['00000001_000.png', '00000001_001.png', '00000001_002.png', '00000003_000.png', '00000003_001.png']
First 5 images in validation set:
['00000002_000.png', '00000010_000.png', '00000012_000.png', '00000014_000.png', '00000017_000.png']


In [6]:
# Cell 6: Define data transforms with selectable augmentation

# Choose augmentation method: 'clahe', 'gamma', 'elastic', or 'none'
augmentation_method = 'clahe'  # Change this as needed

if augmentation_method == 'clahe':
    aug_transform = CLAHETransform(clipLimit=2.0, tileGridSize=(8, 8))
elif augmentation_method == 'gamma':
    aug_transform = GammaCorrectionTransform(gamma=0.8)  # adjust gamma value as needed
elif augmentation_method == 'elastic':
    aug_transform = ElasticTransform(alpha=34, sigma=4)
else:
    aug_transform = None

# Compose training transforms. The pipeline applies the selected augmentation,
# converts the grayscale image to 3 channels, and performs other random augmentations.
train_transforms_list = []
if aug_transform is not None:
    train_transforms_list.append(aug_transform)
    
train_transforms_list += [
    transforms.Grayscale(num_output_channels=3),  # Convert to 3 channels for the model
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]

train_transform = transforms.Compose(train_transforms_list)

# Validation transforms: use deterministic preprocessing only
val_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [7]:
# Cell 7: Create DataLoaders for training and validation
train_dataset = ChestXrayDataset(train_df, root_dir="/student/csc490_project/shared/preprocessed_images/preprocessed_images", transform=train_transform)
val_dataset = ChestXrayDataset(val_df, root_dir="/student/csc490_project/shared/preprocessed_images/preprocessed_images", transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)


In [8]:
# Cell 8: Model List
model_names = ['coatnet_2_rw_224.sw_in12k_ft_in1k', 'convnext_large.fb_in22k', 'densenet121', 'maxvit_rmlp_base_rw_224.sw_in12k_ft_in1k', 'swin_large_patch4_window7_224', 'vgg19.tv_in1k']
num_classes = 15

In [9]:
# Cell 9: Define the loss function selection
loss_type = 'bce'  # Change to 'focal' to use FocalLoss

if loss_type == 'bce':
    criterion = nn.BCEWithLogitsLoss()
elif loss_type == 'focal':
    criterion = FocalLoss(alpha=1, gamma=2, reduction='mean')
else:
    raise ValueError("Invalid loss_type. Choose 'bce' or 'focal'.")


In [10]:
# Cell 10: Training loop with validation AUROC evaluation
for model_name in model_names:
    
    model = timm.create_model(model_name, pretrained=True, num_classes=num_classes)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    if model_name == 'convnext_large.fb_in22k':
        num_epochs = 4
    else:
        num_epochs = 7

    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()
        running_loss = 0.0
        
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * images.size(0)
        
        epoch_loss = running_loss / len(train_dataset)
        epoch_runtime = time.time() - start_time
        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}, Runtime: {epoch_runtime:.2f} seconds")
        
        # Validation evaluation
        model.eval()
        val_loss = 0.0
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                
                preds = torch.sigmoid(outputs).cpu().numpy()
                all_preds.append(preds)
                all_labels.append(labels.cpu().numpy())
        
        val_loss /= len(val_dataset)
        print(f"Model: {model_name}, Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}")
        
        all_preds = np.concatenate(all_preds, axis=0)
        all_labels = np.concatenate(all_labels, axis=0)
        
        # Compute AUROC for each class
        auroc_per_class = {}
        for i, class_name in enumerate(CLASSES):  # Iterate with class names directly
            try:
                auroc = roc_auc_score(all_labels[:, i], all_preds[:, i])
            except ValueError:
                auroc = float('nan')
            auroc_per_class[class_name] = auroc  # Store class name instead of index

        overall_auroc = np.nanmean(list(auroc_per_class.values()))

        print("Validation AUROC per class:")
        for class_name, score in auroc_per_class.items():
            print(f"  {class_name}: {score:.4f}")
        if epoch + 1 != num_epochs:
            print(f"Overall AUROC: {overall_auroc:.4f}\n")
        else:
            print(f"Final AUROC: {overall_auroc:.4f}\n")

    model_save_path = f"/student/csc490_project/shared/new_split_models_no_overlap/CLAHE_{model_name}_model.pth"
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved to {model_save_path}")


Epoch 1/7, Training Loss: 0.1970, Runtime: 642.10 seconds
Model: coatnet_2_rw_224.sw_in12k_ft_in1k, Epoch 1/7, Validation Loss: 0.1960
Validation AUROC per class:
  No Finding: 0.7539
  Atelectasis: 0.7706
  Cardiomegaly: 0.8278
  Effusion: 0.8585
  Infiltration: 0.6873
  Mass: 0.7527
  Nodule: 0.6880
  Pneumonia: 0.7125
  Pneumothorax: 0.8221
  Consolidation: 0.7717
  Edema: 0.8492
  Emphysema: 0.7817
  Fibrosis: 0.7114
  Pleural_Thickening: 0.7380
  Hernia: 0.6855
Overall AUROC: 0.7607

Epoch 2/7, Training Loss: 0.1857, Runtime: 642.24 seconds
Model: coatnet_2_rw_224.sw_in12k_ft_in1k, Epoch 2/7, Validation Loss: 0.1938
Validation AUROC per class:
  No Finding: 0.7557
  Atelectasis: 0.7745
  Cardiomegaly: 0.8624
  Effusion: 0.8582
  Infiltration: 0.6732
  Mass: 0.7627
  Nodule: 0.6699
  Pneumonia: 0.6968
  Pneumothorax: 0.8324
  Consolidation: 0.7739
  Edema: 0.8696
  Emphysema: 0.8188
  Fibrosis: 0.7380
  Pleural_Thickening: 0.7444
  Hernia: 0.7343
Overall AUROC: 0.7710

Epoch 3/7, T